# Transform Test Run

## Set up

TFX requires apache-airflow and docker SDK.

In [1]:
!pip install 'apache-airflow[gcp]' docker tfx

You should consider upgrading via the 'pip install --upgrade pip' command.


In this notebook, we use TFX version 0.13.0

In [2]:
import tfx
tfx.version.__version__

'0.13.0'

TFX requires TensorFlow >= 1.13.1

In [3]:
import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

'1.13.1'

TFX supports Python 3.5 from version 0.13.0

In [4]:
import sys
sys.version

'3.5.2 (default, Nov 12 2018, 13:43:14) \n[GCC 5.4.0 20160609]'

## Download sample data

In [6]:
%%bash
# This enables you to run this notebook twice.
# There should not be train/eval files at ~/taxi/data, since TFX can handle only single file with version 0.13.0
if [ -e ~/taxi/data ]; then
    rm -rf ~/taxi/data
fi

# download taxi data
mkdir -p ~/taxi/data/simple
mkdir -p ~/taxi/serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O ~/taxi/data/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O ~/taxi/taxi_utils.py

--2019-06-14 02:32:08--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922668 (1.8M) [text/plain]
Saving to: ‘/root/taxi/data/simple/data.csv’

     0K .......... .......... .......... .......... ..........  2% 3.18M 1s
    50K .......... .......... .......... .......... ..........  5% 3.59M 1s
   100K .......... .......... .......... .......... ..........  7% 6.40M 0s
   150K .......... .......... .......... .......... .......... 10% 5.24M 0s
   200K .......... .......... .......... .......... .......... 13% 4.28M 0s
   250K .......... .......... .......... .......... .......... 15% 6.39M 0s
   300K .......... .......... .......... .......... .......... 18% 3.01M 0s
   350K ........

## Import

In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from google.protobuf import json_format

from tfx.components.base.base_component import ComponentOutputs
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.orchestration.pipeline import Pipeline
from tfx.orchestration.tfx_runner import TfxRunner
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input
from tfx.utils.channel import Channel
from tfx.utils import types

/usr/local/lib/python3.5/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


## configs

In [8]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')

# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

## Create ExampleGen

In [9]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
train_config = example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
eval_config = example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        train_config,
        eval_config
    ]))

# Create outputs
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root, 'csv_example_gen/train/')

eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root, 'csv_example_gen/eval/')

example_outputs = ComponentOutputs({
    'examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples, eval_examples]
    ),
    'training_examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples]
    ),
    'eval_examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[eval_examples]
    ),    
})

example_gen = CsvExampleGen(
    input_base=examples, # A Channel of 'ExternalPath' type, it contains path of data source.
    output_config=output_config,  # An example_gen_pb2.Output instance, it contains train-eval split ratio.
    outputs=example_outputs # dict from name to output channel, it will be stored example_gen.outputs
)

## Create StatisticsGen

In [10]:
# Create outputs
train_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='train')
train_statistics.uri = os.path.join(_data_root, 'statistics_gen/train/')

eval_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='eval')
eval_statistics.uri = os.path.join(_data_root, 'statistics_gen/eval/')

statistics_outputs = ComponentOutputs({
    'output': Channel(
        type_name='ExampleStatisticsPath',
        static_artifact_collection=[train_statistics, eval_statistics]
    )
})

statistics_gen = StatisticsGen(
    input_data=example_gen.outputs.examples, # A Channel of 'ExamplesPath' type, it is equal to example_outputs
    name='Statistics Generator', # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=statistics_outputs # dict from name to output channel, it will be stored statistics_gen.outputs
)

## Create SchemaGen

In [11]:
# Create outputs
train_schema_path = types.TfxType(type_name='SchemaPath', split='train')
train_schema_path.uri = os.path.join(_data_root, 'schema_gen/')

# NOTE: SchemaGen.executor can handle JUST ONE SchemaPath.
# Two or more SchemaPaths will cause ValueError
# such as "ValueError: expected list length of one but got 2".
schema_outputs = ComponentOutputs({
    'output':Channel(
        type_name='SchemaPath',
        static_artifact_collection=[train_schema_path] 
    )
})

infer_schema = SchemaGen(
    stats=statistics_gen.outputs.output, # A Channel of 'ExampleStatisticsPath' type, it is equal to statistics_outputs
    name='Schema Generator',  # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=schema_outputs # dict from name to output channel, it will be stored schema_gen.outputs
)

## Create Transform

In [12]:
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root,
                                  'transform/transformed_examples/train/')
eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root,
                                 'transform/transformed_examples/eval/')
transform_output = types.TfxType(type_name='TransformPath')
transform_output.uri = os.path.join(_data_root,
                                    'transform/transform_output/')

transform_outputs = ComponentOutputs({
    # Output of 'tf.Transform', which includes an exported 
    # Tensorflow graph suitable for both training and serving
    'transform_output':Channel(
        type_name='TransformPath',
        static_artifact_collection=[transform_output]
    ),
    # transformed_examples: Materialized transformed examples, which includes 
    # both 'train' and 'eval' splits.
    'transformed_examples':Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples, eval_examples]
    )
})

transform = Transform(
    input_data=example_gen.outputs.examples,
    schema=infer_schema.outputs.output,
#     module_file=_taxi_module_file,
    module_file="preprocess.py",
    outputs=transform_outputs
)

## Create pipeline

In [14]:
pipeline = Pipeline(
    pipeline_name="TFX Pipeline",
    pipeline_root=_pipeline_root,
    components=[example_gen, statistics_gen, infer_schema, transform]
)

## Execute

In [15]:
class DirectRunner(TfxRunner):
    """Tfx runner on local"""
    
    def __init__(self, config=None):
        self._config = config or {}
    
    def run(self, pipeline):
        for component in pipeline.components:
            self._execute_component(component)
            
        return pipeline
            
    def _execute_component(self, component):
        input_dict = {key:value.get() for key, value in component.input_dict.items()}
        output_dict = {key: value.get() for key, value in component.outputs.get_all().items()}
        exec_properties = component.exec_properties
        executor = component.executor()
        executor.Do(input_dict, output_dict, exec_properties)

In [16]:
pipeline = DirectRunner().run(pipeline)

INFO:tensorflow:Starting Executor execution.
[2019-06-14 02:32:49,709] {base_executor.py:72} INFO - Starting Executor execution.
INFO:tensorflow:Inputs for Executor is: {"input-base": [{"artifact_type": {"properties": {"state": "STRING", "type_name": "STRING", "span": "INT", "split": "STRING", "name": "STRING"}, "name": "ExternalPath"}, "artifact": {"uri": "/root/taxi/data/simple", "properties": {"type_name": {"stringValue": "ExternalPath"}, "split": {"stringValue": ""}}}}]}
[2019-06-14 02:32:49,713] {base_executor.py:74} INFO - Inputs for Executor is: {"input-base": [{"artifact_type": {"properties": {"state": "STRING", "type_name": "STRING", "span": "INT", "split": "STRING", "name": "STRING"}, "name": "ExternalPath"}, "artifact": {"uri": "/root/taxi/data/simple", "properties": {"type_name": {"stringValue": "ExternalPath"}, "split": {"stringValue": ""}}}}]}
INFO:tensorflow:Outputs for Executor is: {"eval_examples": [{"artifact_type": {"properties": {"state": "STRING", "type_name": "STR

[2019-06-14 02:32:57,474] {fn_api_runner.py:437} INFO - Running (((ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/DoOnce/Read_40)+(ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/InitializeWrite_41))+(ref_PCollection_PCollection_24/Write))+(ref_PCollection_PCollection_25/Write)
[2019-06-14 02:32:57,488] {fn_api_runner.py:437} INFO - Running ((ShuffleSplittrain/ReshufflePerKey/GroupByKey/Read)+(((ref_AppliedPTransform_ShuffleSplittrain/ReshufflePerKey/FlatMap(restore_timestamps)_34)+(ref_AppliedPTransform_ShuffleSplittrain/RemoveRandomKeys_35))+(ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/WriteBundles_42)))+(((ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/Pair_43)+(ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/WindowInto(WindowIntoFn)_44))+(OutputSplittrain/Write/WriteImpl/GroupByKey/Write))
[2019-06-14 02:32:57,634] {tfrecordio.py:57} WARNING - Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as

[2019-06-14 02:33:02,915] {fn_api_runner_transforms.py:490} INFO - ==================== <function sink_flattens at 0x7f5f0b510620> ====================
[2019-06-14 02:33:02,923] {fn_api_runner_transforms.py:490} INFO - ==================== <function greedily_fuse at 0x7f5f0b5106a8> ====================
[2019-06-14 02:33:02,934] {fn_api_runner_transforms.py:490} INFO - ==================== <function read_to_impulse at 0x7f5f0b510730> ====================
[2019-06-14 02:33:02,938] {fn_api_runner_transforms.py:490} INFO - ==================== <function impulse_to_input at 0x7f5f0b5107b8> ====================
[2019-06-14 02:33:02,941] {fn_api_runner_transforms.py:490} INFO - ==================== <function inject_timer_pcollections at 0x7f5f0b510950> ====================
[2019-06-14 02:33:02,945] {fn_api_runner_transforms.py:490} INFO - ==================== <function sort_stages at 0x7f5f0b5109d8> ====================
[2019-06-14 02:33:02,946] {fn_api_runner_transforms.py:490} INFO - ======

[2019-06-14 02:33:07,174] {fn_api_runner.py:437} INFO - Running (((GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/FlattenTopKUniquesResults/Read)+(ref_AppliedPTransform_GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/TopKUniquesStatsGenerator/DeserializeTopKUniquesFeatureStatsProto_168))+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/FlattenFeatureStatistics/Transcode/1))+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/FlattenFeatureStatistics/Write/1)
[2019-06-14 02:33:07,196] {fn_api_runner.py:437} INFO - Running ((GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/FlattenFeatureStatistics/Read)+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/MergeDatasetFeatureStatisticsProtos/Precombine))+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/MergeDatasetFeatureStatisticsProtos/Group/Write)
[201

[2019-06-14 02:33:15,201] {fn_api_runner.py:437} INFO - Running ((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/Flatten/Read)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Precombine))+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Group/Write)
[2019-06-14 02:33:15,364] {fn_api_runner.py:437} INFO - Running ((((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Group/Read)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Merge))+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/ExtractOutputs))+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/FlattenF

...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[2019-06-14 02:33:28,883] {saver.py:1483} INFO - Saver not created because there are no variables in the graph to restore
[2019-06-14 02:33:29,973] {fn_api_runner.py:437} INFO - Running ((AnalyzeDataset/CacheableCombineAccumulate[scale_to_z_score/mean_and_var]/InitialCombineGlobally/CombinePerKey/Group/Read)+(((AnalyzeDataset/CacheableCombineAccumulate[scale_to_z_score/mean_and_var]/InitialCombineGlobally/CombinePerKey/Merge)+(AnalyzeDataset/CacheableCombineAccumulate[scale_to_z_score/mean_and_var]/InitialCombineGlobally/CombinePerKey/ExtractOutputs))+(ref_AppliedPTransform_AnalyzeDataset/CacheableCombineAccumulate[scale_to_z_score/mean_and_var]/InitialCombineGlobally/UnKey_30)))+(((ref_AppliedPTransform_AnalyzeDataset/CacheableCombineMerge[scale_to_z_score/mean_and_var]/MergeCombinesGlobally/KeyWithVoid_33)+(AnalyzeDataset/CacheableCombineMerge[scale_to_z_score/mean_and_var]/MergeCombinesGlobally/

[2019-06-14 02:33:30,605] {fn_api_runner.py:437} INFO - Running (AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/MergeCombinesGlobally/CombinePerKey/Group/Read)+(((AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/MergeCombinesGlobally/CombinePerKey/Merge)+((AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/MergeCombinesGlobally/CombinePerKey/ExtractOutputs)+(ref_AppliedPTransform_AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/MergeCombinesGlobally/UnKey_305)))+(ref_PCollection_PCollection_189/Write))
[2019-06-14 02:33:30,782] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/MergeCombinesGlobally/DoOnce/Read_307)+(ref_AppliedPTransform_AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/MergeCombinesGlobally/InjectDefault_308))+(((ref_AppliedPTransform_AnalyzeDataset/CacheableCombineMerge[bucketize_2/quantiles]/ExtractOutputs/FlatMap(extract_outputs)_310)+(ref_A

[2019-06-14 02:33:31,997] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary/vocabulary]/WriteToFile/Write/WriteImpl/DoOnce/Read_143)+((ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary/vocabulary]/WriteToFile/Write/WriteImpl/InitializeWrite_144)+(ref_PCollection_PCollection_87/Write)))+(ref_PCollection_PCollection_86/Write)
[2019-06-14 02:33:32,019] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary/vocabulary]/Prepare/Read_137)+((ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary/vocabulary]/OrderElements_138)+(ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary/vocabulary]/WriteToFile/Write/WriteImpl/Map(<lambda at iobase.py:984>)_145)))+((ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary/vocabulary]/WriteToFile/Write/Wr

[2019-06-14 02:33:32,562] {filebasedsink.py:290} INFO - Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
[2019-06-14 02:33:32,666] {filebasedsink.py:327} INFO - Renamed 1 shards in 0.10 seconds.
[2019-06-14 02:33:32,691] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary_1/vocabulary]/CreatePath/Read_213)+(ref_AppliedPTransform_AnalyzeDataset/VocabularyWrite[compute_and_apply_vocabulary_1/vocabulary]/WaitForVocabularyFile_214))+((ref_AppliedPTransform_AnalyzeDataset/CreateTensorBinding[compute_and_apply_vocabulary_1/vocabulary/Placeholder]_215)+((AnalyzeDataset/CreateSavedModel/Flatten/Transcode/9)+(AnalyzeDataset/CreateSavedModel/Flatten/Write/9)))
[2019-06-14 02:33:32,725] {fn_api_runner.py:437} INFO - Running (((AnalyzeDataset/CacheableCombineAccumulate[bucketize/quantiles]/InitialCombineGlobally/CombinePerKey/Group/Read)+(AnalyzeDataset/CacheableCombineAccumulate[bucket

[2019-06-14 02:33:36,664] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_Materialize[0]/Write/Write/WriteImpl/DoOnce/Read_405)+((ref_AppliedPTransform_Materialize[0]/Write/Write/WriteImpl/InitializeWrite_406)+(ref_PCollection_PCollection_251/Write)))+(ref_PCollection_PCollection_250/Write)
[2019-06-14 02:33:36,686] {fn_api_runner.py:437} INFO - Running (((((ref_AppliedPTransform_ReadTransformDataset[0]/Read/Read_365)+(ref_AppliedPTransform_ReadTransformDataset[0]/AddKey_366))+((ref_AppliedPTransform_ReadTransformDataset[0]/ParseExamples_367)+((ref_AppliedPTransform_DecodeTransformDataset[0]/ApplyDecodeFn_369)+((ref_AppliedPTransform_TransformDataset[0]/Batch/BatchElements/ParDo(_GlobalWindowsBatchingDoFn)_373)+((ref_AppliedPTransform_TransformDataset[0]/Transform_374)+(ref_AppliedPTransform_TransformDataset[0]/ConvertAndUnbatch_375))))))+((ref_AppliedPTransform_TransformDataset[0]/MakeCheapBarrier_376)+(ref_PCollection_PCollection_234/Write)))+((ref_AppliedPTransform_Enc

## Check Result

In [17]:
!ls -Rlhs /root/taxi/data/simple/

/root/taxi/data/simple/:
total 1.9M
4.0K drwxr-xr-x 4 root root 4.0K Jun 14 02:32 csv_example_gen
1.9M -rw-r--r-- 1 root root 1.9M Jun 14 02:32 data.csv
4.0K drwxr-xr-x 2 root root 4.0K Jun 14 02:33 schema_gen
4.0K drwxr-xr-x 4 root root 4.0K Jun 14 02:33 statistics_gen
4.0K drwxr-xr-x 4 root root 4.0K Jun 14 02:33 transform

/root/taxi/data/simple/csv_example_gen:
total 8.0K
4.0K drwxr-xr-x 2 root root 4.0K Jun 14 02:32 eval
4.0K drwxr-xr-x 2 root root 4.0K Jun 14 02:32 train

/root/taxi/data/simple/csv_example_gen/eval:
total 204K
204K -rw-r--r-- 1 root root 201K Jun 14 02:32 data_tfrecord-00000-of-00001.gz

/root/taxi/data/simple/csv_example_gen/train:
total 408K
408K -rw-r--r-- 1 root root 405K Jun 14 02:32 data_tfrecord-00000-of-00001.gz

/root/taxi/data/simple/schema_gen:
total 8.0K
8.0K -rw-r--r-- 1 root root 4.5K Jun 14 02:33 schema.pbtxt

/root/taxi/data/simple/statistics_gen:
total 8.0K
4.0K drwxr-xr-x 2 root root 4.0K Jun 14 02:33 eval
4.0K drwxr-xr-x 2 root root 4.0K Jun 14

`anomalies.pbtxt` is only simple text file. To visualize it, we are going to;

1. get the path of `anomalies.pbtxt`from `example_validator`
2. parse `anomalies.pbtxt` into anomalies (protobuf)
3. visualize schema with [tfdv](https://www.tensorflow.org/tfx/data_validation/get_started)


In [28]:
# 1. get the path of `transformed_examples/train`
def get_transformed_examples(transform):
    artifacts = transform.outputs.transformed_examples.get()
    return types.get_split_uri(artifacts, 'train')

transformed_examples_dir = get_transformed_examples(transform)

In [33]:
# 2. generate statistics from tfrecord
from pathlib import Path
import tensorflow_data_validation as tfdv

def generate_stats(transformed_examples_dir):
    path = Path(transformed_examples_dir)
    for filepath in path.glob('*'):
        # since we are using python 3.5, not 3.6+
        return tfdv.generate_statistics_from_tfrecord(str(filepath))

stats = generate_stats(transformed_examples_dir)

[2019-06-14 03:24:10,951] {pipeline.py:143} INFO - Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
[2019-06-14 03:24:13,747] {fn_api_runner_transforms.py:490} INFO - ==================== <function annotate_downstream_side_inputs at 0x7f5f0b510268> ====================
[2019-06-14 03:24:13,752] {fn_api_runner_transforms.py:490} INFO - ==================== <function fix_side_input_pcoll_coders at 0x7f5f0b510378> ====================
[2019-06-14 03:24:13,756] {fn_api_runner_transforms.py:490} INFO - ==================== <function lift_combiners at 0x7f5f0b510400> ====================
[2019-06-14 03:24:13,763] {fn_api_runner_transforms.py:490} INFO - ==================== <function expand_sdf at 0x7f5f0b510488> ====================
[2019-06-14 03:24:13,766] {fn_api_runner_transforms.py:490} INFO - ==================== <function expand_gbk at 0x7f5f0b510510> ====================
[2019-06-14 03:24:13,775] {fn_api_runner_transforms.py:490} INFO - ==

[2019-06-14 03:24:20,242] {fn_api_runner.py:437} INFO - Running (GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PreCombineFn)/Group/Read)+((((GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PreCombineFn)/Merge)+(GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PreCombineFn)/ExtractOutputs))+((ref_AppliedPTransform_GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/Map(StripNonce)_21)+(ref_AppliedPTransform_GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/WindowIntoOriginal_22)))+(GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/Flatten/Write/1))
[2019-06-14 03:24:20,539] {fn_api_runner.py:437} INFO - Running ((GenerateStatistics/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/Flatten/Read)+(GenerateS

In [34]:
# 3. visualize statistics with tfdv

tfdv.visualize_statistics(stats)